# Data Loader (Cue Conflict)

In [1]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

dataset_path = '/kaggle/input/cue-conflict/'

transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),        
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) 
])

dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=2)
print(f"Class names: {dataset.classes}")

for images, labels in data_loader:
    print(f"Batch of images: {images.shape}")
    print(f"Batch of labels: {labels}")
    break

Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']
Batch of images: torch.Size([32, 3, 224, 224])
Batch of labels: tensor([ 9,  0, 13,  0, 11,  9, 11, 13,  5,  8,  2, 14,  2, 11,  8,  3,  1, 10,
         6,  2,  9,  6,  1,  3, 14, 10,  7,  7,  4, 13,  3,  4])


# Teacher Models

**VIT**

In [2]:
import torch
from torch import nn
from torchvision.models import vit_b_16

class ViTModel(nn.Module): 
    def __init__(self, num_classes, pretrained=True):
        super(ViTModel, self).__init__()
        self.model = vit_b_16(pretrained=pretrained)
        
        for param in self.model.parameters():
            param.requires_grad = False
            
        in_features = self.model.heads.head.in_features
        self.model.heads.head = nn.Linear(in_features, num_classes)
        for param in self.model.heads.head.parameters():
            param.requires_grad = True

    def forward(self, x):
        return self.model(x)

    def get_model(self):
        return self.model

# num_classes = 10 
# vit_model = ViTModel(num_classes=num_classes, pretrained=True)
# model_instance = vit_model.get_model()
# print(model_instance)

**ResNet50**

In [3]:
import torch
from torchvision import models
import torch.nn as nn

class ResNet50Model(nn.Module):
    def __init__(self, num_classes, pretrained=True):
        super(ResNet50Model, self).__init__()
        self.model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT if pretrained else None)
    
        for param in self.model.parameters():
            param.requires_grad = False
        in_features = self.model.fc.in_features
        self.model.fc = nn.Linear(in_features, num_classes)
        for param in self.model.fc.parameters():
            param.requires_grad = True

    def forward(self, x):
        return self.model(x)
    def get_model(self):
        return self.model

# num_classes = 10 
# resnet_model = ResNet50Model(num_classes=num_classes, pretrained=True)
# model_instance = resnet_model.get_model()
# print(model_instance)


**DenseNet121**

In [4]:
import torch
from torchvision import models
import torch.nn as nn

class DenseNet121Model(nn.Module):
    def __init__(self, num_classes, pretrained=True):
        super(DenseNet121Model, self).__init__()
        self.model = models.densenet121(pretrained=pretrained)

        for param in self.model.features.parameters():
            param.requires_grad = False
        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(in_features, num_classes)
        for param in self.model.classifier.parameters():
            param.requires_grad = True

    def forward(self, x):
        return self.model(x)
    def get_model(self):
        return self.model

# num_classes = 10  
# densenet_model = DenseNet121Model(num_classes=num_classes, pretrained=True)
# model_instance = densenet_model.get_model()
# print(model_instance)

**Vgg-16**

In [5]:
import torch
import torchvision.models as models
import torch.nn as nn

class VGG16Model(nn.Module):
    def __init__(self, num_classes, pretrained=True):
        super(VGG16Model, self).__init__()
        self.model = models.vgg16(pretrained=pretrained)

        for param in self.model.features.parameters():
            param.requires_grad = False
        in_features = self.model.classifier[6].in_features
        self.model.classifier[6] = nn.Linear(in_features, num_classes)
        for param in self.model.classifier[6].parameters():
            param.requires_grad = True

    def forward(self, x):
        return self.model(x)
        
    def get_model(self):
        return self.model
        
# num_classes = 10  
# vgg_model = VGG16Model(num_classes=num_classes, pretrained=True)
# model_instance = vgg_model.get_model()
# print(model_instance)

In [12]:
# num_classes = 16

# vgg16_model = VGG16Model(num_classes=num_classes, pretrained=True).get_model()
# print('VGG16 model loaded')
# resnet50_model = ResNet50Model(num_classes=num_classes, pretrained=True).get_model()
# print('ResNet50 model loaded')
# vit_model = ViTModel(num_classes=num_classes, pretrained=True).get_model()
# print('Vision Transformer (ViT) model loaded')
# densenet121_model = DenseNet121Model(num_classes=num_classes, pretrained=True).get_model()
# print('DenseNet121 model loaded')

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/hashmat/.cache/torch/hub/checkpoints/vgg16-397923af.pth
  1%|          | 6.38M/528M [00:02<03:17, 2.77MB/s]


KeyboardInterrupt: 

# Student Model

In [6]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm
from torchvision.models import vgg11
from torchvision import models

class VGG11(nn.Module):
    def __init__(self, num_classes=16, pretrained=True):
        super(VGG11, self).__init__()

        self.vgg11 = models.vgg11(pretrained=pretrained)
        for param in self.vgg11.features.parameters():
            param.requires_grad = False
            
        self.vgg11.classifier[6] = nn.Linear(self.vgg11.classifier[6].in_features, num_classes)
    
    def forward(self, x):
        return self.vgg11(x)

    def get_model(self):
        return self.vgg11

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg11_model = VGG11(num_classes=16, pretrained=True)
vgg11_model = vgg11_model.to(device)
print('VGG11 model loaded')

VGG11 model loaded


# Dataloader

In [8]:
from torch.utils.data import DataLoader, random_split

def create_data_loaders(dataset_path, batch_size, val_split=0.2, seed=42):
    torch.manual_seed(seed)

    transform = transforms.Compose([
        transforms.Resize((224, 224)), 
        transforms.ToTensor(),        
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    
    dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
    val_size = int(len(dataset) * val_split)
    train_size = len(dataset) - val_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    print(f"Train size: {train_size}, Validation size: {val_size}")
    print(f"Class names: {dataset.classes}")
    
    return train_loader, val_loader

# FineTuning

In [9]:
import torch
import torch.optim as optim
import torch.nn as nn
import os
import zipfile
from tqdm import tqdm

def fine_tune_model(model_class, dataset_path, output_path, num_classes, epochs=10, batch_size=32, learning_rate=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_loader, val_loader = create_data_loaders(dataset_path, batch_size)
    model = model_class(num_classes=num_classes, pretrained=True)
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()

    # Optimizer configuration for ViT and ResNet
    if isinstance(model, ViTModel):  # For ViT
        optimizer = optim.Adam(model.get_model().heads.head.parameters(), lr=learning_rate)
    elif isinstance(model, ResNet50Model):  # For ResNet
        optimizer = optim.Adam(model.get_model().fc.parameters(), lr=learning_rate)  # Use 'fc' for ResNet
    else:  # For other models (e.g., DenseNet, etc.)
        optimizer = optim.Adam(model.get_model().classifier.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        total_samples = 0

        print(f"Epoch {epoch+1}/{epochs}: Training started...")
        for images, labels in tqdm(train_loader, desc="Training", unit="batch"):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += torch.sum(preds == labels)
            total_samples += images.size(0)

        train_loss_avg = train_loss / total_samples
        train_accuracy = (train_correct.double() / total_samples) * 100
        model.eval()
        val_loss = 0.0
        val_correct = 0
        with torch.no_grad():
            print("Validation started...")
            for images, labels in tqdm(val_loader, desc="Validating", unit="batch"):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += torch.sum(preds == labels)

        val_loss_avg = val_loss / len(val_loader.dataset)
        val_accuracy = (val_correct.double() / len(val_loader.dataset)) * 100
        print(f"Epoch {epoch+1}/{epochs}: "
              f"Training Accuracy: {train_accuracy:.4f}, "
              f"Training Loss: {train_loss_avg:.4f} | "
              f"Validation Accuracy: {val_accuracy:.4f}, "
              f"Validation Loss: {val_loss_avg:.4f}")

    # Save the fine-tuned model
    model_name = model_class.__name__.lower()
    model_path = os.path.join(output_path, f"{model_name}_fine_tuned_{num_classes}_classes.pth")
    torch.save(model.get_model().state_dict(), model_path)
    print(f"Model saved at {model_path}")
    
    # Zip the model
    zip_path = model_path.replace(".pth", ".zip")
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(model_path, os.path.basename(model_path))
    os.remove(model_path)
    print(f"Model saved and zipped to {zip_path}")


Fine tuning all models on Labelled Image Net

In [10]:
dataset_path = "/kaggle/input/labelledimagenet"  
output_path = "/kaggle/working/FinetunedModels" 
os.makedirs(output_path, exist_ok=True)
model_class = VGG11
fine_tune_model(model_class, dataset_path, output_path, num_classes=16, epochs=2, batch_size=128, learning_rate=0.001)

Train size: 6029, Validation size: 1507
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']
Epoch 1/1: Training started...


Training: 100%|██████████| 48/48 [00:22<00:00,  2.09batch/s]


Validation started...


Validating: 100%|██████████| 12/12 [00:07<00:00,  1.58batch/s]


Epoch 1/1: Training Accuracy: 79.5488, Training Loss: 0.7620 | Validation Accuracy: 89.5820, Validation Loss: 0.3258
Model saved at /kaggle/working/FinetunedModels/vgg11_fine_tuned_16_classes.pth
Model saved and zipped to /kaggle/working/FinetunedModels/vgg11_fine_tuned_16_classes.zip


In [11]:
dataset_path = "/kaggle/input/labelledimagenet" 
output_path = "/kaggle/working/FinetunedModels"
os.makedirs(output_path, exist_ok=True)
model_class = ViTModel 
fine_tune_model(model_class, dataset_path, output_path, num_classes=16, epochs=10, batch_size=128, learning_rate=0.001)

Train size: 6029, Validation size: 1507
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:01<00:00, 213MB/s] 


Epoch 1/1: Training started...


Training: 100%|██████████| 48/48 [00:37<00:00,  1.27batch/s]


Validation started...


Validating: 100%|██████████| 12/12 [00:09<00:00,  1.21batch/s]


Epoch 1/1: Training Accuracy: 68.3530, Training Loss: 1.5726 | Validation Accuracy: 88.3875, Validation Loss: 0.7454
Model saved at /kaggle/working/FinetunedModels/vitmodel_fine_tuned_16_classes.pth
Model saved and zipped to /kaggle/working/FinetunedModels/vitmodel_fine_tuned_16_classes.zip


In [28]:
dataset_path = "/kaggle/input/labelledimagenet" 
output_path = "/kaggle/working/FinetunedModels"  
os.makedirs(output_path, exist_ok=True)
model_class = VGG16Model  
fine_tune_model(model_class, dataset_path, output_path, num_classes=16, epochs=10, batch_size=128, learning_rate=0.001)

Train size: 6029, Validation size: 1507
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']
Epoch 1/10: Training started...


Training: 100%|██████████| 189/189 [00:22<00:00,  8.52batch/s]


Validation started...


Validating: 100%|██████████| 48/48 [00:05<00:00,  8.45batch/s]


Epoch 1/10: Training Accuracy: 86.0010, Training Loss: 0.7470 | Validation Accuracy: 92.3689, Validation Loss: 0.4543
Epoch 2/10: Training started...


...

Epoch 9/10: Training Accuracy: 98.2418, Training Loss: 1.0731 | Validation Accuracy: 93.5634, Validation Loss: 6.7265
Epoch 10/10: Training started...


Training: 100%|██████████| 189/189 [00:22<00:00,  8.53batch/s]


Validation started...


Validating: 100%|██████████| 48/48 [00:05<00:00,  8.57batch/s]


Epoch 10/10: Training Accuracy: 98.3248, Training Loss: 1.5090 | Validation Accuracy: 93.1652, Validation Loss: 13.5141
Model saved at /kaggle/working/FinetunedModels/vgg16model_fine_tuned_16_classes.pth
Model saved and zipped to /kaggle/working/FinetunedModels/vgg16model_fine_tuned_16_classes.zip


In [33]:
dataset_path = "/kaggle/input/labelledimagenet"  
output_path = "/kaggle/working/FinetunedModels" 
os.makedirs(output_path, exist_ok=True)
model_class = DenseNet121Model  
fine_tune_model(model_class, dataset_path, output_path, num_classes=16, epochs=10, batch_size=128, learning_rate=0.001)

Train size: 6029, Validation size: 1507
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']
Epoch 1/10: Training started...


Training: 100%|██████████| 189/189 [00:21<00:00,  8.94batch/s]


Validation started...


Validating: 100%|██████████| 48/48 [00:05<00:00,  8.41batch/s]


Epoch 1/10: Training Accuracy: 85.7522, Training Loss: 0.5507 | Validation Accuracy: 95.8195, Validation Loss: 0.1706
Epoch 2/10: Training started...


...

Epoch 9/10: Training Accuracy: 99.2702, Training Loss: 0.0317 | Validation Accuracy: 97.0803, Validation Loss: 0.0886
Epoch 10/10: Training started...


Training: 100%|██████████| 189/189 [00:20<00:00,  9.10batch/s]


Validation started...


Validating: 100%|██████████| 48/48 [00:05<00:00,  8.36batch/s]


Epoch 10/10: Training Accuracy: 99.2370, Training Loss: 0.0304 | Validation Accuracy: 97.6775, Validation Loss: 0.0784
Model saved at /kaggle/working/FinetunedModels/densenet121model_fine_tuned_16_classes.pth
Model saved and zipped to /kaggle/working/FinetunedModels/densenet121model_fine_tuned_16_classes.zip


In [48]:
dataset_path = "/kaggle/input/labelledimagenet"  
output_path = "/kaggle/working/FinetunedModels" 
os.makedirs(output_path, exist_ok=True)
model_class = ResNet50Model
fine_tune_model(model_class, dataset_path, output_path, num_classes=16, epochs=10, batch_size=128, learning_rate=0.001)

Train size: 6029, Validation size: 1507
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']
Epoch 1/10: Training started...


Training: 100%|██████████| 189/189 [00:21<00:00,  8.95batch/s]


Validation started...


Validating: 100%|██████████| 48/48 [00:05<00:00,  8.91batch/s]


Epoch 1/10: Training Accuracy: 85.0887, Training Loss: 0.6219 | Validation Accuracy: 96.4167, Validation Loss: 0.2967
Epoch 2/10: Training started...


...

Epoch 9/10: Training Accuracy: 99.8341, Training Loss: 0.0157 | Validation Accuracy: 99.1374, Validation Loss: 0.0511
Epoch 10/10: Training started...


Training: 100%|██████████| 189/189 [00:20<00:00,  9.08batch/s]


Validation started...


Validating: 100%|██████████| 48/48 [00:05<00:00,  8.55batch/s]


Epoch 10/10: Training Accuracy: 99.9005, Training Loss: 0.0117 | Validation Accuracy: 99.0046, Validation Loss: 0.0506
Model saved at /kaggle/working/FinetunedModels/resnet50model_fine_tuned_16_classes.pth
Model saved and zipped to /kaggle/working/FinetunedModels/resnet50model_fine_tuned_16_classes.zip


**FineTuning on Cue Conflict**

In [31]:
dataset_path = "/kaggle/input/cue-conflict"  
output_path = "/kaggle/working/FinetunedModels" 
os.makedirs(output_path, exist_ok=True)
model_class = VGG11
fine_tune_model(model_class, dataset_path, output_path, num_classes=16, epochs=10, batch_size=128, learning_rate=0.001)

Train size: 1024, Validation size: 256
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']
Epoch 1/10: Training started...


Training: 100%|██████████| 8/8 [00:05<00:00,  1.34batch/s]


Validation started...


Validating: 100%|██████████| 2/2 [00:01<00:00,  1.01batch/s]


Epoch 1/10: Training Accuracy: 12.2070, Training Loss: 2.8175 | Validation Accuracy: 27.7344, Validation Loss: 2.2946
Epoch 2/10: Training started...


...

Epoch 9/10: Training Accuracy: 97.5586, Training Loss: 0.0918 | Validation Accuracy: 67.5781, Validation Loss: 1.3885
Epoch 10/10: Training started...


Training: 100%|██████████| 8/8 [00:03<00:00,  2.51batch/s]


Validation started...


Validating: 100%|██████████| 2/2 [00:01<00:00,  1.66batch/s]


Epoch 10/10: Training Accuracy: 97.8516, Training Loss: 0.0663 | Validation Accuracy: 66.0156, Validation Loss: 1.5510
Model saved at /kaggle/working/FinetunedModels/vgg11_fine_tuned_16_classes.pth
Model saved and zipped to /kaggle/working/FinetunedModels/vgg11_fine_tuned_16_classes.zip


In [10]:
dataset_path = "/kaggle/input/cue-conflict"  
output_path = "/kaggle/working/FinetunedModels" 
os.makedirs(output_path, exist_ok=True)
model_class = VGG11
fine_tune_model(model_class, dataset_path, output_path, num_classes=16, epochs=2, batch_size=128, learning_rate=0.001)

Train size: 1024, Validation size: 256
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG11_Weights.IMAGENET1K_V1`. You can also use `weights=VGG11_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg11-8a719046.pth" to /root/.cache/torch/hub/checkpoints/vgg11-8a719046.pth
100%|██████████| 507M/507M [00:02<00:00, 224MB/s]  


Epoch 1/2: Training started...


Training: 100%|██████████| 8/8 [00:05<00:00,  1.55batch/s]


Validation started...


...

Epoch 1/2: Training Accuracy: 12.2070, Training Loss: 2.8175 | Validation Accuracy: 27.7344, Validation Loss: 2.2946
Epoch 2/2: Training started...


Training: 100%|██████████| 8/8 [00:03<00:00,  2.42batch/s]


Validation started...


Validating: 100%|██████████| 2/2 [00:01<00:00,  1.90batch/s]


Epoch 2/2: Training Accuracy: 36.8164, Training Loss: 1.9767 | Validation Accuracy: 43.7500, Validation Loss: 1.6926
Model saved at /kaggle/working/FinetunedModels/vgg11_fine_tuned_16_classes.pth
Model saved and zipped to /kaggle/working/FinetunedModels/vgg11_fine_tuned_16_classes.zip


In [11]:
dataset_path = "/kaggle/input/cue-conflict" 
output_path = "/kaggle/working/FinetunedModels"
os.makedirs(output_path, exist_ok=True)
model_class = ViTModel 
fine_tune_model(model_class, dataset_path, output_path, num_classes=16, epochs=10, batch_size=128, learning_rate=0.001)

Train size: 1024, Validation size: 256
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:01<00:00, 211MB/s]  


Epoch 1/10: Training started...


Training: 100%|██████████| 8/8 [00:06<00:00,  1.15batch/s]


Validation started...


...

Epoch 9/10: Training Accuracy: 83.2031, Training Loss: 1.0280 | Validation Accuracy: 62.1094, Validation Loss: 1.4105
Epoch 10/10: Training started...


Training: 100%|██████████| 8/8 [00:07<00:00,  1.13batch/s]


Validation started...


Validating: 100%|██████████| 2/2 [00:02<00:00,  1.14s/batch]


Epoch 10/10: Training Accuracy: 84.6680, Training Loss: 0.9506 | Validation Accuracy: 62.8906, Validation Loss: 1.3670
Model saved at /kaggle/working/FinetunedModels/vitmodel_fine_tuned_16_classes.pth
Model saved and zipped to /kaggle/working/FinetunedModels/vitmodel_fine_tuned_16_classes.zip


In [12]:
dataset_path = "/kaggle/input/cue-conflict" 
output_path = "/kaggle/working/FinetunedModels"  
os.makedirs(output_path, exist_ok=True)
model_class = VGG16Model  
fine_tune_model(model_class, dataset_path, output_path, num_classes=16, epochs=10, batch_size=128, learning_rate=0.001)

Train size: 1024, Validation size: 256
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 210MB/s] 


Epoch 1/10: Training started...


Training: 100%|██████████| 8/8 [00:03<00:00,  2.23batch/s]


Validation started...


...

Epoch 9/10: Training Accuracy: 95.8008, Training Loss: 0.1193 | Validation Accuracy: 60.9375, Validation Loss: 1.7450
Epoch 10/10: Training started...


Training: 100%|██████████| 8/8 [00:03<00:00,  2.27batch/s]


Validation started...


Validating: 100%|██████████| 2/2 [00:01<00:00,  1.43batch/s]


Epoch 10/10: Training Accuracy: 96.7773, Training Loss: 0.1192 | Validation Accuracy: 59.3750, Validation Loss: 1.8596
Model saved at /kaggle/working/FinetunedModels/vgg16model_fine_tuned_16_classes.pth
Model saved and zipped to /kaggle/working/FinetunedModels/vgg16model_fine_tuned_16_classes.zip


In [13]:
dataset_path = "/kaggle/input/cue-conflict"  
output_path = "/kaggle/working/FinetunedModels" 
os.makedirs(output_path, exist_ok=True)
model_class = DenseNet121Model  
fine_tune_model(model_class, dataset_path, output_path, num_classes=16, epochs=10, batch_size=128, learning_rate=0.001)

Train size: 1024, Validation size: 256
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 155MB/s] 


Epoch 1/10: Training started...


Training: 100%|██████████| 8/8 [00:03<00:00,  2.42batch/s]


Validation started...


...

Epoch 9/10: Training Accuracy: 69.0430, Training Loss: 1.6014 | Validation Accuracy: 42.1875, Validation Loss: 1.9062
Epoch 10/10: Training started...


Training: 100%|██████████| 8/8 [00:02<00:00,  2.69batch/s]


Validation started...


Validating: 100%|██████████| 2/2 [00:01<00:00,  1.68batch/s]


Epoch 10/10: Training Accuracy: 70.6055, Training Loss: 1.5154 | Validation Accuracy: 48.4375, Validation Loss: 1.8573
Model saved at /kaggle/working/FinetunedModels/densenet121model_fine_tuned_16_classes.pth
Model saved and zipped to /kaggle/working/FinetunedModels/densenet121model_fine_tuned_16_classes.zip


In [14]:
dataset_path = "/kaggle/input/cue-conflict"  
output_path = "/kaggle/working/FinetunedModels" 
os.makedirs(output_path, exist_ok=True)
model_class = ResNet50Model
fine_tune_model(model_class, dataset_path, output_path, num_classes=16, epochs=10, batch_size=128, learning_rate=0.001)

Train size: 1024, Validation size: 256
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 185MB/s]


Epoch 1/10: Training started...


Training: 100%|██████████| 8/8 [00:03<00:00,  2.48batch/s]


Validation started...


...

Epoch 9/10: Training Accuracy: 77.9297, Training Loss: 1.5550 | Validation Accuracy: 39.0625, Validation Loss: 2.1348
Epoch 10/10: Training started...


Training: 100%|██████████| 8/8 [00:03<00:00,  2.53batch/s]


Validation started...


Validating: 100%|██████████| 2/2 [00:01<00:00,  1.64batch/s]


Epoch 10/10: Training Accuracy: 80.6641, Training Loss: 1.4662 | Validation Accuracy: 40.6250, Validation Loss: 2.1002
Model saved at /kaggle/working/FinetunedModels/resnet50model_fine_tuned_16_classes.pth
Model saved and zipped to /kaggle/working/FinetunedModels/resnet50model_fine_tuned_16_classes.zip


# Evaluation

In [55]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

def load_fine_tuned_model(model_path, num_classes=16):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = VGG11(num_classes=num_classes, pretrained=False)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    model.eval()
    return model

def evaluate_model(model, dataset_path, batch_size=32):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    transform = transforms.Compose([
        transforms.Resize((224, 224)), 
        transforms.ToTensor(),        
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    eval_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
    eval_loader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    print(f"Evaluating on dataset with {len(eval_dataset)} samples and classes: {eval_dataset.classes}")
    total_samples = 0
    correct_predictions = 0
    
    with torch.no_grad():
        for images, labels in tqdm(eval_loader, desc="Evaluating"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            
            correct_predictions += torch.sum(preds == labels).item()
            total_samples += labels.size(0)
    
    accuracy = (correct_predictions / total_samples)*100
    print(f"Accuracy on the evaluation dataset: {accuracy:.4f} %")
    
    return accuracy

In [12]:
model_path = "/kaggle/working/FinetunedModel/vgg11_fine_tuned_16_classes.pth"  
eval_dataset_path = "/kaggle/input/labelledimagenet"  
model = load_fine_tuned_model(model_path, num_classes=16)
evaluate_model(model, eval_dataset_path, batch_size=32)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_23/4178167424.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recomme

Evaluating on dataset with 7536 samples and classes: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


Evaluating: 100%|██████████| 236/236 [00:26<00:00,  9.03it/s]

Accuracy on the evaluation dataset: 97.2797 %


97.27972399150742

In [25]:
model_path = "/kaggle/working/FinetunedModel/vgg11_fine_tuned_16_classes.pth" 
eval_dataset_path = "/kaggle/input/cue-conflict"  
model = load_fine_tuned_model(model_path, num_classes=16)
evaluate_model(model, eval_dataset_path, batch_size=32)

/tmp/ipykernel_23/4178167424.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


Evaluating on dataset with 1280 samples and classes: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


Evaluating: 100%|██████████| 40/40 [00:03<00:00, 11.09it/s]

Accuracy on the evaluation dataset: 14.1406 %


14.140625000000002

# Logit Matching Knowledge Distillation

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm  

def logit_matching_loss(student_logits, teacher_logits, student_temperature=1.0, teacher_temperature=1.0):
    student_logits = student_logits - student_logits.mean(dim=-1, keepdim=True)
    teacher_logits = teacher_logits - teacher_logits.mean(dim=-1, keepdim=True)
    student_logits = student_logits / student_temperature
    teacher_logits = teacher_logits / teacher_temperature
    loss = F.mse_loss(student_logits, teacher_logits)
    return loss

def distill(model_class, teacher_model_class, train_loader, val_loader, num_classes, 
            epochs=10, batch_size=32, learning_rate=0.001, 
            student_temperature=2.0, teacher_temperature=2.0, alpha=0.5):
   
    student_model = model_class(num_classes=num_classes)  # Pass class reference
    optimizer = optim.Adam(student_model.parameters(), lr=learning_rate)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    student_model = student_model.to(device)
    teacher_model = teacher_model_class(num_classes=num_classes)  # Pass class reference
    teacher_model = teacher_model.to(device)
    teacher_model.eval()
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        student_model.train()
        running_loss = 0.0

        with tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", unit="batch") as pbar:
            for images, labels in pbar:
                images, labels = images.to(device), labels.to(device)
                student_outputs = student_model(images)

                with torch.no_grad():
                    teacher_outputs = teacher_model(images)

                ce_loss = criterion(student_outputs, labels)
                distillation_loss = logit_matching_loss(student_outputs, teacher_outputs, 
                                                        student_temperature=student_temperature,
                                                        teacher_temperature=teacher_temperature)
                total_loss = (1 - alpha) * ce_loss + alpha * distillation_loss
                optimizer.zero_grad()
                total_loss.backward()
                optimizer.step()
                running_loss += total_loss.item()
                pbar.set_postfix(loss=running_loss / len(pbar))

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")

        student_model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = student_model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f"Validation Accuracy at epoch {epoch+1}: {accuracy:.2f}%")

    return student_model

In [28]:
dataset_path = "/kaggle/input/labelledimagenet"
batch_size = 128
train_loader, val_loader = create_data_loaders(dataset_path, batch_size)
teacher_model = VGG11(num_classes=16, pretrained=True) 
student_model_class = VGG11  

distilled_model = distill(
    model_class=student_model_class, 
    teacher_model_class=teacher_model.__class__,  
    train_loader=train_loader,
    val_loader=val_loader,
    num_classes=16,
    epochs=10,
    batch_size=32,
    learning_rate=0.001,
    student_temperature=1.0,
    teacher_temperature=1.0,
    alpha=0.5
)

Train size: 6029, Validation size: 1507
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


Epoch 1/10: 100%|██████████| 48/48 [00:22<00:00,  2.17batch/s, loss=1.08] 

Epoch 1/10, Loss: 1.0766347348690033


Validation Accuracy at epoch 1: 81.09%


Epoch 2/10:  29%|██▉       | 14/48 [00:07<00:19,  1.77batch/s, loss=0.261]


KeyboardInterrupt: 

In [17]:
dataset_path = "/kaggle/input/cue-conflict"
batch_size = 128
train_loader, val_loader = create_data_loaders(dataset_path, batch_size)
teacher_model = VGG16Model(num_classes=16, pretrained=True) 
student_model_class = VGG11  

distilled_model = distill(
    model_class=student_model_class, 
    teacher_model_class=teacher_model.__class__,  
    train_loader=train_loader,
    val_loader=val_loader,
    num_classes=16,
    epochs=8,
    batch_size=32,
    learning_rate=0.001,
    student_temperature=1.0,
    teacher_temperature=1.0,
    alpha=0.5
)

Train size: 1024, Validation size: 256
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


Epoch 1/8: 100%|██████████| 8/8 [00:04<00:00,  1.65batch/s, loss=1.5]  

Epoch 1/8, Loss: 1.4951717108488083


Validation Accuracy at epoch 1: 20.31%


Epoch 2/8: 100%|██████████| 8/8 [00:04<00:00,  1.69batch/s, loss=1.4]  

...

Epoch 7/8, Loss: 1.0239086970686913


Validation Accuracy at epoch 7: 56.64%


Epoch 8/8: 100%|██████████| 8/8 [00:04<00:00,  1.69batch/s, loss=0.947]

Epoch 8/8, Loss: 0.9469453915953636


Validation Accuracy at epoch 8: 66.80%


In [18]:
dataset_path = "/kaggle/input/cue-conflict"
batch_size = 128
train_loader, val_loader = create_data_loaders(dataset_path, batch_size)
teacher_model = DenseNet121Model(num_classes=16, pretrained=True) 
student_model_class = VGG11  

distilled_model = distill(
    model_class=student_model_class, 
    teacher_model_class=teacher_model.__class__,  
    train_loader=train_loader,
    val_loader=val_loader,
    num_classes=16,
    epochs=8,
    batch_size=32,
    learning_rate=0.001,
    student_temperature=1.0,
    teacher_temperature=1.0,
    alpha=0.5
)

Train size: 1024, Validation size: 256
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


Epoch 1/8: 100%|██████████| 8/8 [00:04<00:00,  1.93batch/s, loss=1.55] 

Epoch 1/8, Loss: 1.5523029267787933


Validation Accuracy at epoch 1: 20.70%


Epoch 2/8: 100%|██████████| 8/8 [00:04<00:00,  1.99batch/s, loss=1.43] 

...

Epoch 7/8, Loss: 1.09344881772995


Validation Accuracy at epoch 7: 48.44%


Epoch 8/8: 100%|██████████| 8/8 [00:04<00:00,  1.99batch/s, loss=1.03] 

Epoch 8/8, Loss: 1.031299814581871


Validation Accuracy at epoch 8: 53.52%


In [20]:
dataset_path = "/kaggle/input/cue-conflict"
batch_size = 128
train_loader, val_loader = create_data_loaders(dataset_path, batch_size)
teacher_model = ResNet50Model(num_classes=16, pretrained=True) 
student_model_class = VGG11  

distilled_model = distill(
    model_class=student_model_class, 
    teacher_model_class=teacher_model.__class__,  
    train_loader=train_loader,
    val_loader=val_loader,
    num_classes=16,
    epochs=8,
    batch_size=32,
    learning_rate=0.001,
    student_temperature=1.0,
    teacher_temperature=1.0,
    alpha=0.5
)

Train size: 1024, Validation size: 256
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


Epoch 1/8: 100%|██████████| 8/8 [00:03<00:00,  2.02batch/s, loss=1.48] 

Epoch 1/8, Loss: 1.4757149517536163


Validation Accuracy at epoch 1: 10.55%


Epoch 2/8: 100%|██████████| 8/8 [00:03<00:00,  2.04batch/s, loss=1.41] 

...

Epoch 7/8, Loss: 1.1321796029806137


Validation Accuracy at epoch 7: 56.25%


Epoch 8/8: 100%|██████████| 8/8 [00:03<00:00,  2.04batch/s, loss=1.09] 

Epoch 8/8, Loss: 1.08830127120018


Validation Accuracy at epoch 8: 59.38%


In [21]:
dataset_path = "/kaggle/input/cue-conflict"
batch_size = 128
train_loader, val_loader = create_data_loaders(dataset_path, batch_size)
teacher_model = ViTModel(num_classes=16, pretrained=True) 
student_model_class = VGG11  

distilled_model = distill(
    model_class=student_model_class, 
    teacher_model_class=teacher_model.__class__,  
    train_loader=train_loader,
    val_loader=val_loader,
    num_classes=16,
    epochs=8,
    batch_size=32,
    learning_rate=0.001,
    student_temperature=1.0,
    teacher_temperature=1.0,
    alpha=0.5
)

Train size: 1024, Validation size: 256
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


Epoch 1/8: 100%|██████████| 8/8 [00:08<00:00,  1.02s/batch, loss=1.53] 

Epoch 1/8, Loss: 1.5306316316127777


Validation Accuracy at epoch 1: 18.36%


Epoch 2/8: 100%|██████████| 8/8 [00:08<00:00,  1.02s/batch, loss=1.44] 

...

Epoch 7/8, Loss: 1.0402371436357498


Validation Accuracy at epoch 7: 60.16%


Epoch 8/8: 100%|██████████| 8/8 [00:08<00:00,  1.03s/batch, loss=0.96] 

Epoch 8/8, Loss: 0.9599767252802849


Validation Accuracy at epoch 8: 63.28%


# Multi-Teacher Equal Weighted Logit Matching

In [26]:
import torch
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm  

def logit_matching_loss(student_logits, teacher_logits_list, student_temperature=1.0, teacher_temperature=1.0):
    averaged_teacher_logits = torch.mean(torch.stack(teacher_logits_list), dim=0)
    student_logits = student_logits - student_logits.mean(dim=-1, keepdim=True)
    averaged_teacher_logits = averaged_teacher_logits - averaged_teacher_logits.mean(dim=-1, keepdim=True)
    student_logits = student_logits / student_temperature
    averaged_teacher_logits = averaged_teacher_logits / teacher_temperature
    loss = F.mse_loss(student_logits, averaged_teacher_logits)
    return loss

def multi_teacher_distill(model_class, teacher_models, train_loader, val_loader, num_classes, 
                          epochs=10, batch_size=32, learning_rate=0.001, 
                          student_temperature=2.0, teacher_temperature=2.0, alpha=0.5):
    student_model = model_class(num_classes=num_classes)
    optimizer = optim.Adam(student_model.parameters(), lr=learning_rate)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    student_model.to(device)

    for teacher_model in teacher_models:
        teacher_model.to(device)
        teacher_model.eval()
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(epochs):
        student_model.train()
        running_loss = 0.0
        
        epoch_progress = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", ncols=100, leave=False)

        for images, labels in epoch_progress:
            images, labels = images.to(device), labels.to(device)
            student_outputs = student_model(images)
            
            teacher_logits_list = []
            for teacher_model in teacher_models:
                with torch.no_grad():
                    teacher_logits_list.append(teacher_model(images))
            
            ce_loss = criterion(student_outputs, labels) 
            distillation_loss = logit_matching_loss(student_outputs, teacher_logits_list, 
                                                     student_temperature=student_temperature,
                                                     teacher_temperature=teacher_temperature)
            total_loss = (1 - alpha) * ce_loss + alpha * distillation_loss
            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()
            running_loss += total_loss.item()
            epoch_progress.set_postfix(loss=running_loss / len(epoch_progress))

        print(f"Epoch {epoch+1}/{epochs}, Training Loss: {running_loss / len(train_loader):.4f}")
        student_model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = student_model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Validation Accuracy: {accuracy:.2f}%")
    
    return student_model


In [27]:
 dataset_path = "/kaggle/input/cue-conflict"
batch_size = 128
train_loader, val_loader = create_data_loaders(dataset_path, batch_size)
teacher_model_1 = VGG16Model(num_classes=16, pretrained=True) 
teacher_model_2 = ViTModel(num_classes=16, pretrained=True) 
student_model_class = VGG11  

multi_teacher_distill(
    model_class=student_model_class, 
    teacher_models=[teacher_model_1, teacher_model_2],  # List of teacher models
    train_loader=train_loader, 
    val_loader=val_loader, 
    num_classes=16, 
    epochs=8, 
    batch_size=32, 
    learning_rate=0.001, 
    student_temperature=1.0, 
    teacher_temperature=1.0, 
    alpha=0.5
)

Train size: 1024, Validation size: 256
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


Epoch 1/8, Training Loss: 1.4877


Epoch 1/8, Validation Accuracy: 16.02%


Epoch 2/8, Training Loss: 1.3979


Epoch 2/8, Validation Accuracy: 32.03%


...

Epoch 7/8, Training Loss: 1.0874


Epoch 7/8, Validation Accuracy: 53.91%


Epoch 8/8, Training Loss: 1.0271


Epoch 8/8, Validation Accuracy: 63.67%


VGG11(
  (vgg11): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (12): ReLU(inplace=True)
      (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (14): ReLU(inplace=True)
      (15): MaxPool2d(kernel_size=2,

In [29]:
dataset_path = "/kaggle/input/cue-conflict"
batch_size = 128
train_loader, val_loader = create_data_loaders(dataset_path, batch_size)
teacher_model_1 = VGG16Model(num_classes=16, pretrained=True) 
teacher_model_2 = ViTModel(num_classes=16, pretrained=True) 
teacher_model_3 = ResNet50Model(num_classes=16, pretrained=True) 

student_model_class = VGG11  

multi_teacher_distill(
    model_class=student_model_class, 
    teacher_models=[teacher_model_1, teacher_model_2, teacher_model_3],  # List of teacher models
    train_loader=train_loader, 
    val_loader=val_loader, 
    num_classes=16, 
    epochs=8, 
    batch_size=32, 
    learning_rate=0.001, 
    student_temperature=1.0, 
    teacher_temperature=1.0, 
    alpha=0.5
)

Train size: 1024, Validation size: 256
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


Epoch 1/8, Training Loss: 1.4787


Epoch 1/8, Validation Accuracy: 25.39%


Epoch 2/8, Training Loss: 1.3638


Epoch 2/8, Validation Accuracy: 32.81%


...

Epoch 7/8, Training Loss: 1.0608


Epoch 7/8, Validation Accuracy: 59.77%


Epoch 8/8, Training Loss: 0.9919


Epoch 8/8, Validation Accuracy: 60.55%


VGG11(
  (vgg11): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (12): ReLU(inplace=True)
      (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (14): ReLU(inplace=True)
      (15): MaxPool2d(kernel_size=2,